# Init

In [2]:
import sys
import os
import json
import time
import re

from tqdm import tqdm

cwd = os.getcwd()
os.chdir(cwd)
sys.path.append('tools')

import chat
import parse_data
import sql

# Craft Prompt

In [22]:
system_prompt = ""

prompt_dir = os.path.join(cwd, 'prompt')
version = 'v1.0.0'
fname = f'answer_generator-{version}.md'
prompt_fpath = os.path.join(prompt_dir, fname)

with open(prompt_fpath, 'r') as f:
    prompt_template = ''.join(f.readlines())

def make_prompt(data: dict, idx: int) -> str:

    prompt = prompt_template

    # SQL Query
    sql_generator = data['sql_generator'].get(f'stage_{idx+1}', {})
    if sql_generator:
        sql_query = sql_generator[0]['sql_query']
        reg_p = re.compile('<SQL Query>')
        prompt = re.sub(reg_p, sql_query, prompt)
    else:
        reg_p = re.compile('\n<SQL Query>\n')
        prompt = re.sub(reg_p, '', prompt)
        reg_p = re.compile('\n## SQL Query\n')
        prompt = re.sub(reg_p, '', prompt)

    # SQL Result
    if sql_generator:
        sql_res = str(sql_generator[0]['sql_res'])
        reg_p = re.compile('<SQL Result>')
        prompt = re.sub(reg_p, sql_res, prompt)
    else:
        reg_p = re.compile('\n<SQL Result>\n')
        prompt = re.sub(reg_p, '', prompt)
        reg_p = re.compile('\n## SQL Result\n')
        prompt = re.sub(reg_p, '', prompt)

    # NER Result
    if data['ner']['stage_1']['result']:
        ner_res = [i for i in data['ner']['stage_1']['sql'].values() if i][0]
        ner_res = [i['result'] for i in ner_res if i['result']][0][0]
        ner_res = json.dumps(ner_res, ensure_ascii=False, indent=2)
        reg_p = re.compile('<NER Result>')
        prompt = re.sub(reg_p, ner_res, prompt)
    else:
        reg_p = re.compile('\n<NER Result>\n')
        prompt = re.sub(reg_p, '', prompt)
        reg_p = re.compile('\n## NER Result\n')
        prompt = re.sub(reg_p, '', prompt)

    # replace query
    query = data['team'][idx]['question']
    reg_p = re.compile('<Current Query>')
    prompt = re.sub(reg_p, query, prompt)

    return prompt

In [30]:
question_path = os.path.join(cwd, 'answer_tmp' + os.sep + 'answer-deepseek_v3-2025-01-27.json')

questions = parse_data.read_json(question_path)

# sort the questions by tid
questions = sorted(questions, key=lambda x: int(x['tid'].split('-')[-1]))

# GLM-4-Plus

In [6]:
model = 'glm_4_plus'

## Test

In [29]:
query = make_prompt(questions[24], 2)

history = []

start_time = time.time()
message = chat.create_message(query, history=history, system_prompt=system_prompt, temperature=0.7, top_p=0.9, response_format='text')
end_time = time.time()

execution_time = end_time - start_time
usage = chat.get_token_usage(message, True)
content = chat.get_content(message, True)
history = chat.build_history(history, message=message)

{'prompt_tokens': 353, 'completion_tokens': 10, 'total_tokens': 363}
总营收为176.45亿元。


In [9]:
t = questions[1]
t['answer_generator'] = {}
t['answer_generator']['stage_1']= content
t['token_usage'] = {}
t['token_usage']['answer_generator-stage_1'] = [usage]
t['time_usage'] = {}
t['time_usage']['answer_generator-stage_1'] = [f"{execution_time:.2f}s"]
t = [t]

saved_path = os.path.join(cwd, 'answer_tmp' + os.sep + f'stage_1-{model}-answer_generator-test-{version}.json')
parse_data.write_json(t, saved_path)

## ALL

In [39]:
answers = []

for question in tqdm(questions[77:]):
    res = question
    res['answer_generator'] = {}

    history = []

    try:
        for idx in range(len(question['team'])):
            query = make_prompt(question, idx)

            history.append({'role': 'user', 'content': query})

            start_time = time.time()
            message = chat.create_message(query, history=history, system_prompt=system_prompt, temperature=0.7, top_p=0.9, response_format='text')
            end_time = time.time()

            execution_time = end_time - start_time
            usage = chat.get_token_usage(message, False)
            content = chat.get_content(message, False)

            history = chat.build_history(history, message=message)
            
            res['answer_generator'][f'stage_{idx+1}']= content
            res['token_usage']['answer_generator-stage_{idx+1}'] = [usage]
            res['time_usage']['answer_generator-stage_{idx+1}'] = [f"{execution_time:.2f}s"]
    except:
        pass

    answers.append(res)

saved_path = os.path.join(cwd, 'answer_tmp' + os.sep + f'glm-answer_generator-0127.json')
parse_data.write_json(answers, saved_path)

100%|██████████| 24/24 [05:03<00:00, 12.66s/it]


In [41]:
from openai import OpenAI

deepseek_api = 'sk-ba0f5eed3bea4fa6be16eb33b139c684'

client = OpenAI(api_key= deepseek_api, base_url="https://api.deepseek.com")

In [42]:
answers = []

for question in tqdm(questions[:]):
    res = question
    res['answer_generator'] = {}

    history = []

    for idx in range(len(question['team'])):
        query = make_prompt(question, idx)

        history.append({'role': 'user', 'content': query})

        start_time = time.time()
        response = client.chat.completions.create(
            model="deepseek-chat",
            messages=history,
            stream=False,
            top_p=0.7,
            temperature=0.9
        )
        end_time = time.time()

        response = json.loads(response.to_json())
        content = response['choices'][0]['message']['content']
        usage = response.get('usage', {})
        execution_time = end_time - start_time


        history.append({'role': 'assistant', 'content': content})
        
        res['answer_generator'][f'stage_{idx+1}']= content
        res['token_usage']['answer_generator-stage_{idx+1}'] = [usage]
        res['time_usage']['answer_generator-stage_{idx+1}'] = [f"{execution_time:.2f}s"]

    answers.append(res)

saved_path = os.path.join(cwd, 'answer_tmp' + os.sep + f'deepseek-answer_generator-0127-2.json')
parse_data.write_json(answers, saved_path)

  0%|          | 0/101 [00:38<?, ?it/s]


KeyboardInterrupt: 